# Instalacja Pulp'a

In [ ]:
!pip install pulp
!sudo apt-get install coinor-cbc glpk-utils coinor-clp

In [ ]:
### kod sprawdza czy instalacja się udała
import pulp
print(pulp.__version__)

In [ ]:
pulp.pulpTestAll()

# Zadania:

1. przeczytaj materiały dotyczące pulp ze strony internetowej https://coin-or.github.io/pulp/main/index.html oraz przejdź przez blending problem (przepisz do swojego notebooka)

2. w blending problem dodaj zmienną decyzyjną. Stwórz zmienną która pojawi się w końcowym składzie produktu (tj. wartość zmiennej będzie miała niezerową postać)

3. Dodaj ograniczenie co do zawartości witaminy B 

  Zawartość:

| Nazwa      | zawartość ug/100g |
|------------|-------------------|
| chicken    | 0.3               |
| beef       | 2.6               |
| mutton     | 2.6               |
| rice       | 0                 |
| wheat bran | 0                 |
| gel        | 0                 |
| sardines   | 8.9               |


Kot ma zjeść minimum 0.24 ug na kilogram masy więc około 0.8 ug dziennie

4. Na końcu ma pojawiać się zdanie podsumowujące wynik, w stylu:
Końcowy produkt będzie kosztować {}, będzie miał w składzie {nazwa}: {} %, {nastepna nazwa}: {} %

## Whiskas blending problem
**Docs:** https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html

**Opis problemu:**

Produkcja karmy dla kotów tak tanio, jak to tylko możliwe, jednocześnie zapewniając, że spełniaja ona określone wymagania dotyczące analizy wartości odżywczej przedstawione na puszkach. Używanymi składnikami są kurczak, wołowina, baranina, ryż, pszenica, żel i sardynki.

### Simplified Formulation

**Zmienne decyzyjne:**

x1 = procent kurczaka w puszce karmy

x2 = procent wołowiny w puszce karmy

x3 = procent sardynek w puszce karmy

**Funkcja celu:**

Minimalizacja kosztów składników dla puszki

$min(0.013x1 + 0.008x2 + 0.007x3)$

**Ograniczenia:**

*   Procentówa suma składników == 100%
*   Proteiny >= 8g/100g
*   Tłuszcz >= 6g/100g
*   Błonnik <= 2g/100g
*   Sól <= 0.4g/100g
*   Witamina B >= 0.8ug/100g

In [33]:
from pulp import *
prob = LpProblem("The_Whiskas_Problem",LpMinimize)
LpVariable("example", None, 100)

x1=LpVariable("Chicken",0,None)
x2=LpVariable("Beef",0,None)
x3=LpVariable("Sardines",0,None)
prob += 0.013*x1 + 0.008*x2 + 0.007*x3, "Total Cost of Ingredients per can"

prob += x1 + x2 + x3 == 100, "PercentagesSum"
prob += 0.100*x1 + 0.200*x2 + 0.25*x3 >= 8.0, "ProteinRequirement"
prob += 0.080*x1 + 0.100*x2 + 0.11*x3 >= 6.0, "FatRequirement"
prob += 0.001*x1 + 0.005*x2 + 0.001*x3 <= 2.0, "FibreRequirement"
prob += 0.002*x1 + 0.005*x2 + 0.005*x3 <= 0.4, "SaltRequirement"
prob += 0.3*x1 + 2.6*x2 + 8.9*x3 >= 0.8, "BVitaminRequirement"

prob.writeLP("WhiskasModel.lp")
prob.solve()

varsdict = {}

print("Status:", LpStatus[prob.status])
for v in prob.variables():
    varsdict[v.name] = str(round(v.varValue,2)) + "%"
    #print(v.name, "=", v.varValue)

#!cat "WhiskasModel.lp"

print("Końcowy produkt będzie kosztować {}, będzie miał w składzie: {}".format(round(value(prob.objective),2),varsdict))

Status: Optimal
Końcowy produkt będzie kosztować 0.9, będzie miał w składzie: {'Beef': '0.0%', 'Chicken': '33.33%', 'Sardines': '66.67%'}


### Full Formulation

**Zmienne decyzyjne:**

x1 = procent kurczaka w puszce karmy

x2 = procent wołowiny w puszce karmy

x3 = procent baraniny w puszce karmy

x4 = procent ryżu w puszce karmy

x5 = procent pszenicy w puszce karmy

x6 = procent żelu w puszce karmy

x7 = procent sardynek w puszce karmy

**Funkcja celu:**

Minimalizacja kosztów składników dla puszki

$min(0.013x1 + 0.008x2 + 0.010x3 + 0.002x4 + 0.005x5 + 0.001x6 + 0.007x7)$

**Ograniczenia:**

*   Procentówa suma składników == 100%
*   Proteiny >= 8g/100g
*   Tłuszcz >= 6g/100g
*   Błonnik <= 2g/100g
*   Sól <= 0.4g/100g
*   Witamina B >= 0.8ug/100g

In [34]:
from pulp import *
prob = LpProblem("The_Whiskas_Problem_2", LpMinimize)

# Creates a list of the Ingredients
Ingredients = ['CHICKEN', 'BEEF', 'MUTTON', 'RICE', 'WHEAT', 'GEL', 'SARDINES']

# A dictionary of the costs of each of the Ingredients is created
costs = {'CHICKEN': 0.013, 
         'BEEF': 0.008, 
         'MUTTON': 0.010, 
         'RICE': 0.002, 
         'WHEAT': 0.005, 
         'GEL': 0.001,
         'SARDINES': 0.007}

# A dictionary of the protein percent in each of the Ingredients is created
proteinPercent = {'CHICKEN': 0.100, 
                  'BEEF': 0.200, 
                  'MUTTON': 0.150, 
                  'RICE': 0.000, 
                  'WHEAT': 0.040, 
                  'GEL': 0.000,
                  'SARDINES': 0.25}

# A dictionary of the fat percent in each of the Ingredients is created
fatPercent = {'CHICKEN': 0.080, 
              'BEEF': 0.100, 
              'MUTTON': 0.110, 
              'RICE': 0.010, 
              'WHEAT': 0.010, 
              'GEL': 0.000,
              'SARDINES': 0.11}

# A dictionary of the fibre percent in each of the Ingredients is created
fibrePercent = {'CHICKEN': 0.001, 
                'BEEF': 0.005, 
                'MUTTON': 0.003, 
                'RICE': 0.100, 
                'WHEAT': 0.150, 
                'GEL': 0.000,
                'SARDINES': 0.001}

# A dictionary of the salt percent in each of the Ingredients is created
saltPercent = {'CHICKEN': 0.002, 
               'BEEF': 0.005, 
               'MUTTON': 0.007, 
               'RICE': 0.002, 
               'WHEAT': 0.008, 
               'GEL': 0.000,
               'SARDINES': 0.005}

BVitaminPercent = {'CHICKEN': 0.3, 
                   'BEEF': 2.6, 
                   'MUTTON': 2.6, 
                   'RICE': 0, 
                   'WHEAT': 0, 
                   'GEL': 0,
                   'SARDINES': 8.9}

ingredient_vars = LpVariable.dicts("",Ingredients,0)
prob += lpSum([costs[i]*ingredient_vars[i] for i in Ingredients]), "Total Cost of Ingredients per can"

prob += lpSum([ingredient_vars[i] for i in Ingredients]) == 100, "PercentagesSum"
prob += lpSum([proteinPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 8.0, "ProteinRequirement"
prob += lpSum([fatPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 6.0, "FatRequirement"
prob += lpSum([fibrePercent[i] * ingredient_vars[i] for i in Ingredients]) <= 2.0, "FibreRequirement"
prob += lpSum([saltPercent[i] * ingredient_vars[i] for i in Ingredients]) <= 0.4, "SaltRequirement"
prob += lpSum([BVitaminPercent[i] * ingredient_vars[i] for i in Ingredients]) >= 0.8, "BVitaminRequirement"

prob.writeLP("WhiskasModel2.lp")
prob.solve()

varsdict = {}

print("Status:", LpStatus[prob.status])
for v in prob.variables():
    varsdict[v.name] = str(round(v.varValue,2)) + "%"
    #print(v.name, "=", v.varValue)

#!cat "WhiskasModel2.lp"

print("Końcowy produkt będzie kosztować {}, będzie miał w składzie: {}".format(round(value(prob.objective),2),varsdict))

Status: Optimal
Końcowy produkt będzie kosztować 0.43, będzie miał w składzie: {'_BEEF': '0.0%', '_CHICKEN': '0.0%', '_GEL': '45.45%', '_MUTTON': '0.0%', '_RICE': '0.0%', '_SARDINES': '54.55%', '_WHEAT': '0.0%'}
